In [1]:
import requests
import base64
import cv2
import json
from matplotlib import pyplot as plt
import numpy as np

In [2]:
def sketch_thick(img_path):
    target_url = "http://localhost:8899/sketch_thick"

#     print(img_path)
    
    dict = {}
    dict['path'] = img_path

    response = requests.post(target_url, data=json.dumps(dict))

    data = json.loads(response.text)
    if 'output_path' in data:
        src = data['output_path']

        img = cv2.imread(src)

        return img, src
    return None, None
def sketch_thin(img_path):
    target_url = "http://localhost:8899/sketch_thin"

    dict = {}
    dict['path'] = img_path

    response = requests.post(target_url, data=json.dumps(dict))

    data = json.loads(response.text)
    if 'output_path' in data:
        src = data['output_path']

        img = cv2.imread(src)

        return img, src
    return None, None

In [3]:
def img_to_imgpath(img):
    path = '/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/test_post.png'
    cv2.imwrite(path, img)
    
    return path

In [4]:
def cartoonize(img):
    target_url = "http://localhost:8894/predict"

    _, img_encoded = cv2.imencode('.jpg', img, params=[cv2.IMWRITE_JPEG_QUALITY, 50])
    img = cv2.imdecode(img_encoded, 1)
    # send http request with image and receive response
    jpg_as_text = base64.b64encode(img_encoded).decode()
    dict = {}
    dict['image'] = jpg_as_text

    response = requests.post(target_url, data=json.dumps(dict))

    lists = json.loads(response.text)
    jpg_original = base64.b64decode(lists[0]['image'])
    jpg_as_np = np.frombuffer(jpg_original, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)

    path = img_to_imgpath(img)
    
    return img, path

In [5]:
def remove_bg(img_path):
    target_url = "http://localhost:8899/remove_bg"

    dict = {}
    dict['img_path'] = img_path

    response = requests.post(target_url, data=json.dumps(dict))

    data = json.loads(response.text)
#     print(data)

    img = cv2.imread(data)
    return img, data

In [ ]:
import os
import shutil

ori_path = glob.glob("/home/ubuntu/workspace/kobaco/dataset/data_processing/kobaco_keyframe/**/*.png")
dst_path = glob.glob("/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/train_data/gt_draw/*.png")

paths = []
for op in ori_path:
    paths.append(op.split('/')[-1])

for dp in dst_path:
    dst_file = dp.split('/')[-1]
    if dst_file in paths:
        idx = paths.index(dst_file)
        src = ori_path[idx]
#         ori_img = cv2.imread(src)
#         dst_img = cv2.imread(dp)
        
        out_dir = '/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/train_data/im_draw/'
        dst = os.path.join(out_dir, dst_file[:-4] + '.jpg')
        
        shutil.copy(src, dst)

In [60]:
import glob
from PIL import Image
import numpy as np
import os
import cv2

# impath = glob.glob("/home/ubuntu/workspace/kobaco/dataset/data_processing/kobaco_keyframe/**/*.png")
impath = glob.glob("/home//ubuntu/workspace/kobaco/sketchy/U-2-Net/train_data/im2/*")
impath = impath[:]
# print(impath)
# impath = impath[1501:]
cnt = 0

for path in impath:
#     print(path)
    output_dir_im= '/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/data_/im/'
    if(not os.path.exists(output_dir_im)):
        os.mkdir(output_dir_im)

    output_dir_gt = '/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/data_/gt2/'
    if(not os.path.exists(output_dir_gt)):
        os.mkdir(output_dir_gt)

#     output_dir_sk = '/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/data_/processed_sk/'
#     if(not os.path.exists(output_dir_sk)):
#         os.mkdir(output_dir_sk)
        
#     output_dir_sk_nobg = '/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/data_/processed_sk_nobg/'
#     if(not os.path.exists(output_dir_sk_nobg)):
#         os.mkdir(output_dir_sk_nobg)
        
#     output_dir_ori = '/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/data_/processed_ori/'
#     if(not os.path.exists(output_dir_ori)):
#         os.mkdir(output_dir_ori)
    
#     output_dir_ori_nobg = '/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/data_/processed_ori_nobg/'
#     if(not os.path.exists(output_dir_ori_nobg)):
#         os.mkdir(output_dir_ori_nobg)
    
    dst = path.split('/')[-1]
    
    img = cv2.imread(path)
    
    img_nobg, path_nobg = remove_bg(path)

    img_gray = cv2.cvtColor(img_nobg, cv2.COLOR_BGR2GRAY)
    
    ret, mask = cv2.threshold(img_gray, 250, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    mask_inv = cv2.bitwise_not(mask)

    # dst_bg = cv2.bitwise_and(dst_img_copy, dst_img_copy, mask=mask_inv)

    
    
    img_sk, path_sk = sketch_thin(path)
    
    if img_sk is None:
        continue
    img_nobg_sk, path_nobg_sk = sketch_thick(path_nobg)
    
    if img_nobg_sk is None:
        continue
    
    k = np.ones((2,2), np.uint8)
    img_nobg_sk = cv2.erode(img_nobg_sk, k)
    
    img_fg = cv2.bitwise_and(img_sk, img_sk, mask=mask)
    img_bg = cv2.bitwise_and(img_nobg_sk, img_nobg_sk, mask=mask_inv)
    
    img_result = img_fg + img_bg
    
#     plt.imshow(img)
#     plt.show()
#     plt.imshow(img_result)
#     plt.show()
    cv2.imwrite(output_dir_im + dst, img)
    cv2.imwrite(output_dir_gt + dst[:-4]+'.png', img_result)
#     cv2.imwrite(output_dir_ori + dst, img)
#     cv2.imwrite(output_dir_ori_nobg + dst, img_nobg)
    cnt += 1
#     if cnt == 1:
#         break

In [39]:
img_gray[100]

array([254, 254, 255, ..., 226, 226, 226], dtype=uint8)

In [12]:
import glob
import os
from PIL import Image
import numpy as np
impath = glob.glob("/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/train_data/gt_draw_newface_inv/*.jpg")
for path in impath:
    output_dir = '/home/ubuntu/workspace/kobaco/sketchy/U-2-Net/train_data/gt_draw_newface_erode'
    if(not os.path.exists(output_dir)):
        os.mkdir(output_dir)
    
    dst = os.path.join(output_dir, path.split('/')[-1][:-4] + '.png')
    
    img = cv2.imread(path)

    k = np.ones((2,2), np.uint8)
    erosion = cv2.erode(img, k)
    
    erosion = 255 - erosion
    
    cv2.imwrite(dst, erosion)
    # plt.imshow(mask,cmap='gray')
    # plt.show()
    # plt.imshow(img)
    # plt.show()
    #     k = np.ones((2,2), np.uint8)
#     erosion = cv2.dilate(img_nobg_sk, k, iterations = 2)
#     k = np.ones((2,2), np.uint8)
#     erosion = cv2.erode(erosion, k)


    # 날카로움 연산 적용
#     blr = cv2.GaussianBlur(erosion, (0, 0), 2)
#     img_nobg_sk = np.clip(2.0*erosion - blr, 0, 255).astype(np.uint8)
    
#     plt.imshow(img)
#     plt.show()
#     plt.imshow(img_nobg)
#     plt.show()
#     plt.imshow(img_sk)
#     plt.show()
#     plt.imshow(img_nobg_sk)
#     plt.show()

In [ ]:
!nvidia-smi

In [45]:


img_paths = glob.glob('./train_data/gt_draw_newface_inv/*')
n = 2
cv2.imwrite('/home/ubuntu/workspace/1.png', cv2.imread(img_paths[n]))
# img = cv2.bitwise_and(cv2.bitwise_not(sketch_thick('/home/ubuntu/workspace/1.png')[0]), cartoonize(cv2.imread(img_paths[0])))
img = sketch_thick('/home/ubuntu/workspace/1.png')[0]
img_n = cv2.bitwise_not(img)


kernel = np.ones((3, 3), np.uint8)
result = cv2.dilate(img_n, kernel, iterations = 1)
result = cv2.bitwise_not(result)
# kernel = np.ones((3, 3), np.uint8)
# result = cv2.erode(result, kernel, iterations = 1)

kernel = np.ones((3, 3), np.uint8)
result = cv2.morphologyEx(result, cv2.MORPH_OPEN, kernel)

Image.fromarray(result)


# k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
# # 침식 연산 적용
# erosion = cv2.erode(img_nobg_sk, k)

# # 날카로움 연산 적용
# blr = cv2.GaussianBlur(erosion, (0, 0), 2)
# img_nobg_sk = np.clip(2.0*erosion - blr, 0, 255).astype(np.uint8)

error: OpenCV(4.5.3) /tmp/pip-req-build-afu9cjzs/opencv/modules/imgproc/src/morph.dispatch.cpp:1022: error: (-215:Assertion failed) !src.empty() in function 'dilate'


In [36]:
img_nobg_sk

In [ ]:
a = glob.glob("")